# Task 1 
In this task, ConvNext model will be used to perform multi-class classification. First a config file will be created containing the hyperparameters of the experiment. Then a script will be executed to initiate the training process and the model checkpoint will be saved. Finally, ROC curve and AUC score will be calculated. 

Dataset has been split in the following way:
- Train set and val set are obtained from the train directory, a random subset sampler is used to split the data in the train directory in 80:20 ratio
- The test set is created from the val directory

## Create config file

In [1]:
conf = """


exp:
    data_dir: /media/saitomar/Work/Projects/DeepLense_Test/task_1_dataset/dataset
    wandb: False
    wandb_api_key: 
    proj_name: DeepLense
    exp_dir: ./runs
    exp_name: Task1-ConvNext-0.1
    device: auto
    log_freq: 20  # steps
    log_to_file: False
    log_to_stdout: True
    val_freq: 1   # epochs
    n_workers: 1
    pin_memory: True
    cache: 2
    task: t1
entity: saitomar

hparams:
    restore_ckpt:
    seed: 0
    batch_size: 16
    start_epoch: 0
    n_epochs: 2
    l_smooth: 0.1
    device: auto
    num_classes: 3

    
    model:
        type : ConvNext
        ConvNext:
            in_channels : 1
            stem_features : 64
            depths : [3, 4, 6]
            widths : [256, 512, 1024]
            num_classes : 3
    optimizer:
        opt_type: adamw
        opt_kwargs:
          lr: 0.005
          
    
    scheduler:
        n_warmup: 1
        max_epochs: 2
        scheduler_type: cosine_annealing

"""

with open("sample_config/config.yaml", "w+") as f:
    f.write(conf)

In [7]:
!python3 pipeline.py --conf sample_config/config.yaml

Set seed 0
Using settings:
 entity: saitomar
exp:
  cache: 2
  data_dir: /media/saitomar/Work/Projects/DeepLense_Test/task_1_dataset/dataset
  device: &id001 !!python/object/apply:torch.device
  - cuda
  exp_dir: ./runs
  exp_name: Task1-ConvNext-0.1
  log_freq: 20
  log_to_file: false
  log_to_stdout: true
  n_workers: 1
  pin_memory: true
  proj_name: DeepLense
  save_dir: ./runs/Task1-ConvNext-0.1
  task: t1
  val_freq: 1
  wandb: false
  wandb_api_key: null
hparams:
  batch_size: 16
  device: *id001
  l_smooth: 0.1
  model:
    ConvNext:
      depths:
      - 3
      - 4
      - 6
      in_channels: 1
      num_classes: 3
      stem_features: 64
      widths:
      - 256
      - 512
      - 1024
    type: ConvNext
  n_epochs: 2
  num_classes: 3
  optimizer:
    opt_kwargs:
      lr: 0.005
    opt_type: adamw
  restore_ckpt: null
  scheduler:
    max_epochs: 2
    n_warmup: 1
    scheduler_type: cosine_annealing
  seed: 0
  start_epoch: 0

Created model with 63500611 parameters.
Ini